# Задание 1

Установите подключение к тестовой БД (содержит данные о мире Игры Престолов)

In [ ]:
!pip install neo4j-driver
!pip install py2neo
!pip install igraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j-driver: filename=neo4j_driver-5.27.0-py3-none-any.whl size=301902 sha256=143000ae2b71f4f4aaaec38fd45751d3577e79fa3a8adf916f95fe112e4ee057
  Stored in directory: /root/.cache/pip/wheels/97/9c/63/9b5a51b678fce6807d92992ffa75382bacc8df2a5016efc986
Successfully built neo4j-driver
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.6 MB/s eta 0:00:00


In [ ]:
import py2neo
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://44.213.108.47:7687",
  auth=basic_auth("neo4j", "desires-grove-fence"))

session = driver.session()

query2=''' LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-all-edges.csv" AS row
MERGE (src:Character {name: row.Source})
MERGE (tgt:Character {name: row.Target})
// global relationship
MERGE (src)-[r:INTERACTS]->(tgt) ON CREATE SET r.weight = toInteger(row.weight)'''

result = session.run(query2,
  parameters={})

ServiceUnavailable: Couldn't connect to 44.213.108.47:7687 (resolved to ()):
Timed out trying to establish connection to ResolvedIPv4Address(('44.213.108.47', 7687))

# 2 Задание

Определите, сколько персонажей упоминаются в БД

In [ ]:
query = '''
MATCH (c:Character)
RETURN count(c) AS total_characters
'''
result = session.run(query)
print("Количество персонажей:", result.single()["total_characters"])

Количество персонажей: 798


# Задание 3

Рассчитайте минимальное, максимальное, среднее значения числа персонажей, с которыми взаимодействуют персонажи из п.2

In [ ]:
query = '''
MATCH (c:Character)-[:INTERACTS]-()
WITH c, count(*) AS interaction_count
RETURN
  min(interaction_count) AS min_interactions,
  max(interaction_count) AS max_interactions,
  avg(interaction_count) AS avg_interactions
'''
result = session.run(query)
data = result.single()
print("Минимум:", data["min_interactions"])
print("Максимум:", data["max_interactions"])
print("Среднее:", data["avg_interactions"])

Минимум: 1
Максимум: 122
Среднее: 7.077694235588974


# Задание 4

Определите персонажей, которые взаимодействуют более чем с 10 другими персонажами

In [ ]:
query = '''
MATCH (c:Character)-[:INTERACTS]-()
WITH c, count(*) AS interaction_count
WHERE interaction_count > 10
RETURN c.name AS character, interaction_count
ORDER BY interaction_count DESC
'''
result = session.run(query)
for record in result:
    print(f"{record['character']}: {record['interaction_count']} взаимодействий")

Tyrion-Lannister: 122 взаимодействий
Jon-Snow: 114 взаимодействий
Jaime-Lannister: 101 взаимодействий
Cersei-Lannister: 97 взаимодействий
Stannis-Baratheon: 89 взаимодействий
Arya-Stark: 84 взаимодействий
Catelyn-Stark: 75 взаимодействий
Sansa-Stark: 75 взаимодействий
Robb-Stark: 74 взаимодействий
Eddard-Stark: 74 взаимодействий
Daenerys-Targaryen: 73 взаимодействий
Joffrey-Baratheon: 69 взаимодействий
Theon-Greyjoy: 66 взаимодействий
Robert-Baratheon: 65 взаимодействий
Bran-Stark: 54 взаимодействий
Tywin-Lannister: 48 взаимодействий
Petyr-Baelish: 47 взаимодействий
Brienne-of-Tarth: 45 взаимодействий
Samwell-Tarly: 44 взаимодействий
Barristan-Selmy: 41 взаимодействий
Renly-Baratheon: 38 взаимодействий
Sandor-Clegane: 36 взаимодействий
Varys: 34 взаимодействий
Tommen-Baratheon: 33 взаимодействий
Mance-Rayder: 33 взаимодействий
Davos-Seaworth: 33 взаимодействий
Gregor-Clegane: 33 взаимодействий
Rodrik-Cassel: 30 взаимодействий
Margaery-Tyrell: 30 взаимодействий
Jeor-Mormont: 28 взаимоде

# Задание 5

Определите диаметр графа взаимодействия персонажей (кратчайший путь во всем графе)

In [ ]:
from py2neo import Graph
from igraph import Graph as IGraph

graph = Graph("bolt://44.213.108.47:7687", auth=("neo4j", "desires-grove-fence"))

query = '''
MATCH (c1:Character)-[r:INTERACTS]-(c2:Character)
RETURN c1.name AS source, c2.name AS target, r.weight AS weight
'''
edges = graph.run(query).data()

ig = IGraph.TupleList([(edge['source'], edge['target']) for edge in edges], directed=False)

diameter = ig.diameter()
print(f"Диаметр графа взаимодействий: {diameter}")

Диаметр графа взаимодействий: 9


# Задание 6

Определите кратчайший путь между Недом Старком и Кхалом Дрого

In [ ]:
try:
    with driver.session() as session:
        res = session.run('''
MATCH (ned:Character {name: "Ned Stark"}), (khal:Character {name: "Khal Drogo"})
MATCH p=shortestPath((ned)-[:INTERACTS*]-(khal))
RETURN p
''', parameters={})

        data = res.data()
        if data:
            print("Кратчайший путь найден:")
            for record in data:
                path = record['p']
                nodes = [node['name'] for node in path if isinstance(node, dict) and 'name' in node]
                print(" => ".join(nodes))
        else:
            print("Кратчайший путь не найден.")
finally:
    driver.close()

<ipython-input-11-1ca7466f3d31>:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Кратчайший путь найден:
Ned Stark => Khal Drogo


# Задание 7

Определите степень центральности для каждого персонажа (количество связей, которые имеет узел в сети)

In [ ]:
driver = GraphDatabase.driver(
  "bolt://44.213.108.47:7687",
  auth=basic_auth("neo4j", "desires-grove-fence"))

session = driver.session()

query = '''
MATCH (c:Character)-[:INTERACTS]-()
WITH c, count(*) AS degree_centrality
RETURN c.name AS character, degree_centrality
ORDER BY degree_centrality DESC
'''
result = session.run(query)
for record in result:
    print(f"{record['character']}: {record['degree_centrality']}")

Tyrion-Lannister: 122
Jon-Snow: 114
Jaime-Lannister: 101
Cersei-Lannister: 97
Stannis-Baratheon: 89
Arya-Stark: 84
Catelyn-Stark: 75
Sansa-Stark: 75
Robb-Stark: 74
Eddard-Stark: 74
Daenerys-Targaryen: 73
Joffrey-Baratheon: 69
Theon-Greyjoy: 66
Robert-Baratheon: 65
Bran-Stark: 54
Tywin-Lannister: 48
Petyr-Baelish: 47
Brienne-of-Tarth: 45
Samwell-Tarly: 44
Barristan-Selmy: 41
Renly-Baratheon: 38
Sandor-Clegane: 36
Varys: 34
Tommen-Baratheon: 33
Mance-Rayder: 33
Davos-Seaworth: 33
Gregor-Clegane: 33
Rodrik-Cassel: 30
Margaery-Tyrell: 30
Jeor-Mormont: 28
Loras-Tyrell: 28
Roose-Bolton: 28
Pycelle: 27
Aemon-Targaryen-(Maester-Aemon): 26
Jorah-Mormont: 26
Asha-Greyjoy: 25
Bronn: 25
Edmure-Tully: 24
Victarion-Greyjoy: 24
Lysa-Arryn: 24
Janos-Slynt: 23
Mace-Tyrell: 23
Melisandre: 23
Ramsay-Snow: 23
Myrcella-Baratheon: 22
Hizdahr-zo-Loraq: 22
Ilyn-Payne: 22
Luwin: 22
Jory-Cassel: 21
Meryn-Trant: 21
Doran-Martell: 21
Drogo: 21
Eddison-Tollett: 20
Gendry: 20
Oberyn-Martell: 20
Robert-Arryn: 20
Amo

# Задание 8

Определите центральность близости  (величину, обратную среднему расстоянию до всех остальных символов в сети) в графе взаимодействия персонажей.  

In [ ]:
driver = GraphDatabase.driver(
    "bolt://44.213.108.47:7687",
    auth=basic_auth("neo4j", "desires-grove-fence")
)

try:
    with driver.session() as session:
        session.run('''
        CALL gds.graph.drop('characterGraph', false) YIELD graphName
        ''')

        session.run('''
        CALL gds.graph.project(
            'characterGraph',
            'Character',
            'INTERACTS'
        );
        ''')

        query_closeness_centrality = '''
        CALL gds.closeness.stream('characterGraph')
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS character_name, score AS closeness_centrality
        ORDER BY closeness_centrality DESC;
        '''
        result = session.run(query_closeness_centrality)
        print("Центральность близости персонажей:")
        for record in result:
            print(f"{record['character_name']}: {record['closeness_centrality']}")
finally:
    driver.close()

Центральность близости персонажей:
Aemon-Targaryen-(Maester-Aemon): 1.0
Arianne-Martell: 1.0
Arthur-Dayne: 1.0
Arya-Stark: 1.0
Asha-Greyjoy: 1.0
Axell-Florent: 1.0
Baelor-Blacktyde: 1.0
Barristan-Selmy: 1.0
Benedar-Belmore: 1.0
Brandon-Stark: 1.0
Bronn: 1.0
Caleotte: 1.0
Cletus-Yronwood: 1.0
Cortnay-Penrose: 1.0
Craster: 1.0
Dunstan-Drumm: 1.0
Ebben: 1.0
Eldon-Estermont: 1.0
Elinor-Tyrell: 1.0
Emrick: 1.0
Franklyn-Flowers: 1.0
Gariss: 1.0
Gilwood-Hunter: 1.0
Goghor-the-Giant: 1.0
Gorne: 1.0
Gorold-Goodbrother: 1.0
Grigg: 1.0
Grunt: 1.0
Guyard-Morrigen: 1.0
Gynir: 1.0
Hairy-Hal: 1.0
Haldon: 1.0
Marya-Seaworth: 1.0
Meldred-Merlyn: 1.0
Morrec: 1.0
Murenmure: 1.0
Patchface: 1.0
Rhaenys-Targaryen: 1.0
Roger-Ryswell: 1.0
Stygg: 1.0
Visenya-Targaryen: 1.0
Rhaenys-Targaryen-(daughter-of-Rhaegar): 1.0
Emmond: 1.0
Gormond-Goodbrother: 1.0
Greydon-Goodbrother: 1.0
Rus: 1.0
Denys-Darklyn: 1.0
Elbert-Arryn: 1.0
Owen-Merryweather: 1.0
Rossart: 1.0
Gelmarr: 1.0
Dancy: 1.0
Elys-Waynwood: 1.0
Gerald-Go

# Задание 9

Определите сообщества (кластеры) взаимодействующих между собой персонажей

In [ ]:
driver = GraphDatabase.driver(
    "bolt://44.213.108.47:7687",
    auth=basic_auth("neo4j", "desires-grove-fence")
)

try:
    with driver.session() as session:
        session.run('''
        CALL gds.graph.drop('characterGraph', false) YIELD graphName
        ''')

        session.run('''
        CALL gds.graph.project(
            'characterGraph',
            'Character',
            {
                INTERACTS: {
                    orientation: 'UNDIRECTED'
                }
            }
        );
        ''')

        l_query = '''
        CALL gds.louvain.write('characterGraph', {
            writeProperty: 'community'
        })
        YIELD communityCount, modularity
        RETURN communityCount, modularity;
        '''
        result = session.run(l_query)
        for rec in result:
            print(f"Количество сообществ: {rec['communityCount']}, Модульность: {rec['modularity']}")

        query_communities = '''
        MATCH (c:Character)
        RETURN c.name AS character_name, c.community AS community
        ORDER BY community
        '''
        community_result = session.run(query_communities)
        print("Персонажи и их сообщества:")
        for rec in community_result:
            print(f"{rec['character_name']}: Сообщество {rec['community']}")
finally:
    driver.close()

Количество сообществ: 11, Модульность: 0.5928451696907928
Персонажи и их сообщества:
Andrey-Dalt: Сообщество 103
Areo-Hotah: Сообщество 103
Arianne-Martell: Сообщество 103
Arys-Oakheart: Сообщество 103
Caleotte: Сообщество 103
Cedra: Сообщество 103
Daemon-Sand: Сообщество 103
Doran-Martell: Сообщество 103
Ellaria-Sand: Сообщество 103
Garin-(orphan): Сообщество 103
Gerold-Dayne: Сообщество 103
Manfrey-Martell: Сообщество 103
Mellei: Сообщество 103
Myrcella-Baratheon: Сообщество 103
Nymeria-Sand: Сообщество 103
Obara-Sand: Сообщество 103
Sarella-Sand: Сообщество 103
Sylva-Santagar: Сообщество 103
Tyene-Sand: Сообщество 103
Franklyn-Fowler: Сообщество 103
Morra: Сообщество 103
Ricasso: Сообщество 103
Timoth: Сообщество 103
Roose-Ryswell: Сообщество 153
Salladhor-Saan: Сообщество 153
Selyse-Florent: Сообщество 153
Shireen-Baratheon: Сообщество 153
Stannis-Baratheon: Сообщество 153
Sybell-Spicer: Сообщество 153
Theomore: Сообщество 153
Wyman-Manderly: Сообщество 153
Gerald-Gower: Сообщество

# Задание 10

Определите мощность сообщества, в которое входит Тирион Ланистер

In [ ]:
driver = GraphDatabase.driver(
    "bolt://44.213.108.47:7687",
    auth=basic_auth("neo4j", "desires-grove-fence")
)

try:
    with driver.session() as session:
        query_find_community = '''
        MATCH (t:Character {name: "Tyrion-Lannister"})
        RETURN t.community AS community
        '''
        result = session.run(query_find_community)
        community = None
        for record in result:
            community = record["community"]

        if community is not None:
            query_community_size = '''
            MATCH (c:Character)
            WHERE c.community = $community
            RETURN COUNT(c) AS community_size
            '''
            result = session.run(query_community_size, community=community)
            for record in result:
                print(f"Мощность сообщества: {record['community_size']}")
finally:
    driver.close()

Мощность сообщества: 128
